## MCP ##

How LLM get access to context (tools and resources)
MCP client host in the LLM app and MCP server (tool, data resources, prompt template)

REST APIs: Standardize how web apps interact with the backend
LSP: Standardize how IDEs interact with language-specific tools
MCP: Standardize how AI apps interact with external systems

# Communication Lifecycle
MCP Client <------> MCP Sever
1. Initialization (init request, init response, init notification)
2. Message Exchange (request, response, notification)
3. Termination

MCP Transport: Handles the underlying mechs of how messages are sent back & fort
1. stdio (standard input output): local severs
2. HTTP+SSE and Streamable HTTP: for remote severs ( needs stateful connection for HTTP+SSE while Streamable HTTP allows both stateful and stateless connection)

-> Streambale HTTP
    POST /mcp to initialize request
    GET /mcp with Accept: text/event-stream   sever can send request to client
    DELETE /mcp to terminate session



In [21]:
import arxiv
import json
import os
from typing import List
from dotenv import load_dotenv
import anthropic


In [22]:
PAPER_DIR = "papers"

'''
search for relevant papers on arxiv based on a topic
store the paper info in a JSON file: title, authors, summary, and link
'''

def search_paper(topic:str, max_results:int=5) -> List[str]:
    client = arxiv.Client()
    search = arxiv.Search(
        query=topic,
        max_results=max_results,
        sort_by=arxiv.SortCriterion.Relevance
    )

    papers = client.results(search)

    path = os.path.join(PAPER_DIR, topic.lower().replace(' ', '_'))
    os.makedirs(path, exist_ok=True)

    file_path = os.path.join(path, "papers_info.json")

    try:
        with open(file_path, 'r') as json_file:
            papers_info = json.load(json_file)
    except (FileNotFoundError, json.JSONDecodeError):
        papers_info = {}

    paper_ids = []
    for paper in papers:
        paper_ids.append(paper.get_short_id())
        paper_info = {
            "title": paper.title,
            "authors": [author.name for author in paper.authors],
            "summary": paper.summary,
            "pdf_url": paper.pdf_url,
            "published": str(paper.published.date())
        }
        papers_info[paper.get_short_id()] = paper_info

    with open(file_path, 'w') as json_file:
        json.dump(papers_info, json_file, indent=2)
    print(f"Saved paper info to: {file_path}")        
    
    return paper_ids

In [23]:
search_paper("machine learning")

Saved paper info to: papers\machine_learning\papers_info.json


['1909.03550v1',
 '1811.04422v1',
 '1707.04849v1',
 '1909.09246v1',
 '2301.09753v1']

In [24]:
def extract_info(paper_id: str) -> str:
    ''' 
    search for information about a paper across all topic directories in the PAPER_DIR
    '''
    for item in os.listdir(PAPER_DIR):
        item_path = os.path.join(PAPER_DIR, item)
        if os.path.exists(item_path):
            file_path = os.path.join(item_path, "papers_info.json")
            if os.path.isfile(file_path):
                try:
                    with open(file_path, 'r') as json_file:
                        papers_info = json.load(json_file)
                        if paper_id in papers_info:
                            return json.dumps(papers_info[paper_id], indent=2)
                except (FileNotFoundError, json.JSONDecodeError) as e:
                    print(f"Error reading {file_path}: {e}")
                    continue
    return f"Paper with ID {paper_id} not found in any topic directory."

In [25]:
extract_info('1909.03550v1')

'{\n  "title": "Lecture Notes: Optimization for Machine Learning",\n  "authors": [\n    "Elad Hazan"\n  ],\n  "summary": "Lecture notes on optimization for machine learning, derived from a course at\\nPrinceton University and tutorials given in MLSS, Buenos Aires, as well as\\nSimons Foundation, Berkeley.",\n  "pdf_url": "http://arxiv.org/pdf/1909.03550v1",\n  "published": "2019-09-08"\n}'

In [26]:
import PyPDF2
import requests
from io import BytesIO
import re

In [27]:
def download_and_extract_pdf_text(pdf_url: str) -> str:
    """
    Download a PDF from a URL and extract its text content
    """
    try:
        # Download the PDF
        response = requests.get(pdf_url, timeout=30)
        response.raise_for_status()
        
        # Read PDF content
        pdf_file = BytesIO(response.content)
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        
        # Extract text from all pages
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text() + "\n"
        
        return text.strip()
    except Exception as e:
        return f"Error downloading or extracting PDF: {str(e)}"

In [28]:
def summarize_paper(paper_id: str) -> str:
    """
    Get comprehensive summary of a paper including abstract, problem statement, 
    methodology, existing solutions, improvements, results, and key findings
    """
    # First get the paper info
    paper_info = None
    for item in os.listdir(PAPER_DIR):
        item_path = os.path.join(PAPER_DIR, item)
        if os.path.exists(item_path):
            file_path = os.path.join(item_path, "papers_info.json")
            if os.path.isfile(file_path):
                try:
                    with open(file_path, 'r') as json_file:
                        papers_info = json.load(json_file)
                        if paper_id in papers_info:
                            paper_info = papers_info[paper_id]
                            break
                except (FileNotFoundError, json.JSONDecodeError) as e:
                    continue
    
    if not paper_info:
        return f"Paper with ID {paper_id} not found in any topic directory."
    
    # Extract PDF text
    pdf_text = download_and_extract_pdf_text(paper_info['pdf_url'])
    
    if pdf_text.startswith("Error"):
        return f"Could not download PDF: {pdf_text}"
    
    # Prepare the prompt for Claude
    prompt = f"""
    Please provide a comprehensive summary of this research paper with the following structure:

    **Paper Title:** {paper_info['title']}
    **Authors:** {', '.join(paper_info['authors'])}
    **Published:** {paper_info['published']}

    Based on the full paper text below, please extract and summarize:

    1. **Abstract/Summary:** Brief overview of the paper
    2. **Problem Statement:** What problem are they trying to solve?
    3. **Methodology/Framework:** What methods, techniques, or frameworks do they use?
    4. **Existing Solutions:** What previous work exists in this area? What are the limitations of existing approaches?
    5. **Proposed Improvements:** How does their approach improve upon existing solutions?
    6. **Results:** What are the main experimental results and findings?
    7. **Key Contributions:** What are the most important contributions and insights?
    8. **Limitations:** What are the acknowledged limitations or potential weaknesses?

    Full Paper Text:
    {pdf_text[:15000]}  # Limit text to avoid token limits
    """
    
    try:
        response = client.messages.create(
            max_tokens=3000,
            model='claude-3-5-sonnet-20241022',
            messages=[{'role': 'user', 'content': prompt}]
        )
        
        return response.content[0].text
    except Exception as e:
        return f"Error generating summary: {str(e)}"

In [29]:
def analyze_papers_by_topic(topic: str) -> str:
    """
    Analyze all papers in a topic directory and provide comparative insights
    """
    topic_dir = topic.lower().replace(' ', '_')
    topic_path = os.path.join(PAPER_DIR, topic_dir)
    
    if not os.path.exists(topic_path):
        return f"No papers found for topic: {topic}"
    
    file_path = os.path.join(topic_path, "papers_info.json")
    
    try:
        with open(file_path, 'r') as json_file:
            papers_info = json.load(json_file)
    except (FileNotFoundError, json.JSONDecodeError):
        return f"No papers data found for topic: {topic}"
    
    if not papers_info:
        return f"No papers found for topic: {topic}"
    
    # Create a summary of all papers
    papers_summary = []
    for paper_id, info in papers_info.items():
        papers_summary.append(f"""
        Paper ID: {paper_id}
        Title: {info['title']}
        Authors: {', '.join(info['authors'])}
        Published: {info['published']}
        Summary: {info['summary']}
        """)
    
    prompt = f"""
    Please analyze these {len(papers_info)} research papers on the topic of "{topic}" and provide:

    1. **Common Themes:** What are the main research themes and areas of focus?
    2. **Methodological Approaches:** What different methodologies and techniques are being used?
    3. **Research Gaps:** What gaps or opportunities for future research do you identify?
    4. **Key Trends:** What trends do you notice in the research over time?
    5. **Comparative Analysis:** How do these papers relate to each other? Are there complementary or competing approaches?
    6. **Practical Applications:** What are the potential real-world applications of this research?

    Papers Summary:
    {"".join(papers_summary)}
    """
    
    try:
        response = client.messages.create(
            max_tokens=2500,
            model='claude-3-5-sonnet-20241022',
            messages=[{'role': 'user', 'content': prompt}]
        )
        
        return response.content[0].text
    except Exception as e:
        return f"Error analyzing papers: {str(e)}"

In [30]:
def save_summary_to_file(paper_id: str, summary_type: str = "comprehensive") -> str:
    """
    Save paper summary to a text file
    
    Args:
        paper_id: The ID of the paper to summarize and save
        summary_type: Type of summary - "comprehensive" (full analysis) or "basic" (just paper info)
    
    Returns:
        Status message about the save operation
    """
    try:
        # Create summaries directory if it doesn't exist
        summaries_dir = "summaries"
        os.makedirs(summaries_dir, exist_ok=True)
        
        # Get the summary based on type
        if summary_type.lower() == "comprehensive":
            summary_content = summarize_paper(paper_id)
            if summary_content.startswith("Paper with ID") or summary_content.startswith("Could not download"):
                return summary_content
            file_suffix = "_comprehensive_summary"
        elif summary_type.lower() == "basic":
            summary_content = extract_info(paper_id)
            if summary_content.startswith("Paper with ID"):
                return summary_content
            file_suffix = "_basic_info"
        else:
            return "Invalid summary_type. Use 'comprehensive' or 'basic'"
        
        # Create filename
        filename = f"{paper_id}{file_suffix}.txt"
        file_path = os.path.join(summaries_dir, filename)
        
        # Add timestamp and metadata
        from datetime import datetime
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        header = f"""
================================================================================
PAPER SUMMARY - {summary_type.upper()}
================================================================================
Paper ID: {paper_id}
Generated: {timestamp}
Summary Type: {summary_type.capitalize()}
================================================================================

"""
        
        # Write to file
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(header)
            f.write(summary_content)
            f.write(f"\n\n{'='*80}\nEnd of Summary\n{'='*80}")
        
        return f"Summary saved successfully to: {file_path}"
        
    except Exception as e:
        return f"Error saving summary: {str(e)}"

In [31]:
def save_topic_analysis_to_file(topic: str) -> str:
    """
    Save topic analysis to a text file
    
    Args:
        topic: The topic to analyze and save
    
    Returns:
        Status message about the save operation
    """
    try:
        # Create summaries directory if it doesn't exist
        summaries_dir = "summaries"
        os.makedirs(summaries_dir, exist_ok=True)
        
        # Get the topic analysis
        analysis_content = analyze_papers_by_topic(topic)
        
        if analysis_content.startswith("No papers found"):
            return analysis_content
        
        # Create filename
        topic_safe = topic.lower().replace(' ', '_').replace('/', '_')
        filename = f"{topic_safe}_topic_analysis.txt"
        file_path = os.path.join(summaries_dir, filename)
        
        # Add timestamp and metadata
        from datetime import datetime
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        header = f"""
================================================================================
TOPIC ANALYSIS REPORT
================================================================================
Topic: {topic}
Generated: {timestamp}
Analysis Type: Comparative Topic Analysis
================================================================================

"""
        
        # Write to file
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(header)
            f.write(analysis_content)
            f.write(f"\n\n{'='*80}\nEnd of Analysis\n{'='*80}")
        
        return f"Topic analysis saved successfully to: {file_path}"
        
    except Exception as e:
        return f"Error saving topic analysis: {str(e)}"

In [32]:
# Tool Schema for the LLM

tools = [
    {
        "name": "search_papers",
        "description": "Search for relevant papers on arxiv based on a topic",
        "input_schema": {
            "type": "object",
            "properties": {
                "topic": {
                    "type": "string",
                    "description": "The topic to search for papers on arxiv"
                },
                "max_results": {
                    "type": "integer",
                    "description": "Maximum number of results to return",
                    "default": 5
                }
            },
            "required": ["topic"]
        }   
    },
    {
        "name": "extract_info",
        "description": "Extract basic information about a paper by its ID",
        "input_schema": {
            "type": "object",
            "properties": {
                "paper_id": {
                    "type": "string",
                    "description": "The ID of the paper to extract information for"
                }
            },
            "required": ["paper_id"]
        }
    },
    {
        "name": "summarize_paper",
        "description": "Get comprehensive summary of a paper including abstract, problem statement, methodology, existing solutions, improvements, results, and key findings",
        "input_schema": {
            "type": "object",
            "properties": {
                "paper_id": {
                    "type": "string",
                    "description": "The ID of the paper to summarize"
                }
            },
            "required": ["paper_id"]
        }
    },
    {
        "name": "analyze_papers_by_topic",
        "description": "Analyze all papers in a topic directory and provide comparative insights including common themes, methodological approaches, research gaps, and trends",
        "input_schema": {
            "type": "object",
            "properties": {
                "topic": {
                    "type": "string",
                    "description": "The topic to analyze papers for"
                }
            },
            "required": ["topic"]
        }
    },
    {
        "name": "save_summary_to_file",
        "description": "Save paper summary to a text file. Can save either comprehensive analysis or basic paper information",
        "input_schema": {
            "type": "object",
            "properties": {
                "paper_id": {
                    "type": "string",
                    "description": "The ID of the paper to summarize and save"
                },
                "summary_type": {
                    "type": "string",
                    "description": "Type of summary to save: 'comprehensive' (full analysis) or 'basic' (just paper info)",
                    "enum": ["comprehensive", "basic"],
                    "default": "comprehensive"
                }
            },
            "required": ["paper_id"]
        }
    },
    {
        "name": "save_topic_analysis_to_file",
        "description": "Save topic analysis to a text file with comparative insights for all papers in a topic",
        "input_schema": {
            "type": "object",
            "properties": {
                "topic": {
                    "type": "string",
                    "description": "The topic to analyze and save"
                }
            },
            "required": ["topic"]
        }
    }
]

In [33]:
# Tool Mapping and execution

mapping_tool_functions = {
    "search_papers": search_paper,
    "extract_info": extract_info,
    "summarize_paper": summarize_paper,
    "analyze_papers_by_topic": analyze_papers_by_topic,
    "save_summary_to_file": save_summary_to_file,
    "save_topic_analysis_to_file": save_topic_analysis_to_file
}

def execute_tool(tool_name: str, tool_args):
    result = mapping_tool_functions[tool_name](**tool_args)
    if result is None:
        result = "Completed successfully, but no result returned."
    elif isinstance(result, list):
        result = ", ".join(result)
    elif isinstance(result, dict):
        result = json.dumps(result, indent=2)
    else:
        result = str(result)
    return result

In [ ]:
ANTHROPIC_API_KEY = "" # Replace with your actual API key or create a .env file with the key

In [35]:
load_dotenv() 
client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)

In [36]:
'''
response = client.messages.create(max_tokens = 500,
                                  model = 'claude-opus-4-20250514', 
                                    temperature=1,
                                  system="You are a world class poet. Respond only with poems.",
                                  messages = [{'role': 'user', 'content': [{"type":"text",
                                                                            "text": "Why is the ocean salty?"}]}])
'''

'\nresponse = client.messages.create(max_tokens = 500,\n                                  model = \'claude-opus-4-20250514\', \n                                    temperature=1,\n                                  system="You are a world class poet. Respond only with poems.",\n                                  messages = [{\'role\': \'user\', \'content\': [{"type":"text",\n                                                                            "text": "Why is the ocean salty?"}]}])\n'

In [37]:
#print(response.content)

In [38]:
#Querry Looop

def process_query(query):
    
    messages = [{'role': 'user', 'content': query}]
    
    response = client.messages.create(max_tokens = 2024,
                                  model = 'claude-3-7-sonnet-20250219', 
                                  tools = tools,
                                  messages = messages)
    
    process_query = True
    while process_query:
        assistant_content = []

        for content in response.content:
            if content.type == 'text':
                
                print(content.text)
                assistant_content.append(content)
                
                if len(response.content) == 1:
                    process_query = False
            
            elif content.type == 'tool_use':
                
                assistant_content.append(content)
                messages.append({'role': 'assistant', 'content': assistant_content})
                
                tool_id = content.id
                tool_args = content.input
                tool_name = content.name
                print(f"Calling tool {tool_name} with args {tool_args}")
                
                result = execute_tool(tool_name, tool_args)
                messages.append({"role": "user", 
                                  "content": [
                                      {
                                          "type": "tool_result",
                                          "tool_use_id": tool_id,
                                          "content": result
                                      }
                                  ]
                                })
                response = client.messages.create(max_tokens = 2024,
                                  model = 'claude-3-7-sonnet-20250219', 
                                  tools = tools,
                                  messages = messages) 
                
                if len(response.content) == 1 and response.content[0].type == "text":
                    print(response.content[0].text)
                    process_query = False

In [39]:
# Chat Loop

def chat_loop():
    print("Type your queries or 'quit' to exit.")
    while True:
        try:
            query = input("\nQuery: ").strip()
            if query.lower() == 'quit':
                break
    
            process_query(query)
            print("\n")
        except Exception as e:
            print(f"\nError: {str(e)}")

In [40]:
chat_loop()

Type your queries or 'quit' to exit.
I'll search for the latest papers on generative machine learning for you. Let me retrieve that information.
Calling tool search_papers with args {'topic': 'generative machine learning', 'max_results': 5}
I'll search for the latest papers on generative machine learning for you. Let me retrieve that information.
Calling tool search_papers with args {'topic': 'generative machine learning', 'max_results': 5}
Saved paper info to: papers\generative_machine_learning\papers_info.json
Saved paper info to: papers\generative_machine_learning\papers_info.json
I've found the 5 most relevant papers on generative machine learning. Let me provide you with the basic information about each of them:
Calling tool extract_info with args {'paper_id': '2001.04942v2'}
I've found the 5 most relevant papers on generative machine learning. Let me provide you with the basic information about each of them:
Calling tool extract_info with args {'paper_id': '2001.04942v2'}
Calling

In [42]:
print("\nAvailable tools:")
for tool in tools:
    print(f"- {tool['name']}: {tool['description']}")



Available tools:
- search_papers: Search for relevant papers on arxiv based on a topic
- extract_info: Extract basic information about a paper by its ID
- summarize_paper: Get comprehensive summary of a paper including abstract, problem statement, methodology, existing solutions, improvements, results, and key findings
- analyze_papers_by_topic: Analyze all papers in a topic directory and provide comparative insights including common themes, methodological approaches, research gaps, and trends
- save_summary_to_file: Save paper summary to a text file. Can save either comprehensive analysis or basic paper information
- save_topic_analysis_to_file: Save topic analysis to a text file with comparative insights for all papers in a topic
